In [3]:
import torch
from torch import nn
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
from smokingml.datasets.nursing_dataset_v1.dataloading import load_one_session, load_one_windowed_session
from smokingml.datasets.nursing_dataset_v1.utils import window_session, train_test_split_windows

# 1st LSTM
- entire session is sequence of inputs
- each input is one data point (xyz)

$L =$ `len(session)`

$H_{in} = 3$

In [16]:
data_dir = Path('../data/nursingv1_dataset/')
device = 'cuda:0'

In [35]:
class LSTM1(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=3, hidden_size=64, bias=False)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )

    def forward(self, x):
        # x is len(session) x 3
        o, (h,c) = self.lstm(x) # o is shape (len(session), 64)
        logits = self.mlp(o[-1])

        return logits

model = LSTM1().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [38]:
X,y = load_one_session(data_dir, 0)
logits = model(X.to(device).float())

tensor([-0.2104], device='cuda:0', grad_fn=<AddBackward0>)